In [80]:
import numpy as np
import pandas as pd
import operator
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from treelib import Node, Tree


In [81]:
class Node:
    def __init__(self, name, feature=None, threshold=None, left_child1=None, left_child2=None, right_child1=None, right_child2=None, is_leaf=False, value=-1):
        self.feature = feature
        self.threshold = threshold
        self.left_child1 = left_child1
        self.left_child2 = left_child2
        self.right_child1 = right_child1
        self.right_child2 = right_child2
        self.is_leaf = is_leaf
        self.value = value
        self.name = name
    

In [82]:
class MyDecisionTree:
    def __init__(self, min_samples=1):
        self.root_node = Node('root')
        self.node_count = 0
        self.min_samples = min_samples
    
    def predict(self, X):
        
        Y_pred = []
        for x_i in X:

            curr_node = self.root_node
            predicted_y = np.array(self.predict_util(x_i, curr_node))
            # print(predicted_y)
            Y_pred.append(predicted_y)

        return np.array(Y_pred)

    def predict_util(self, x, curr_node):
        
        if(curr_node.is_leaf):
            return curr_node.value
        if x[curr_node.feature[0]] <= curr_node.threshold[0] and x[curr_node.feature[1]] <= curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.left_child1)

        if x[curr_node.feature[0]] > curr_node.threshold[0] and x[curr_node.feature[1]] <= curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.left_child2)

        if x[curr_node.feature[0]] <= curr_node.threshold[0] and x[curr_node.feature[1]] > curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.right_child1)

        elif x[curr_node.feature[0]] > curr_node.threshold[0] and x[curr_node.feature[1]] > curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.right_child2)

    def fit(self, X, Y):
        self.fit_util(X, Y, self.root_node)


    def fit_util(self, X, Y, current_node):
        if np.unique(Y).shape[0] == 1:
            current_node.is_leaf = True
            unq, counts = np.unique(Y, return_counts=True)
            max_freq_idx = np.argmax(counts).flatten()
            current_node.value = unq[max_freq_idx].squeeze()
            current_node.name = f'leaf {current_node.value} {self.node_count}'
            self.node_count += 1
            return
            
        scores = {}
        for column1 in X.columns:
            for column2 in X.columns:

                column = [column1, column2]
                X_train = X[column].to_numpy()
                X_train = X_train.reshape(X_train.shape[0], 2)
                clf = LogisticRegression(random_state=0, max_iter=100)
                clf.fit(X_train, Y)
                score = clf.score(X_train, Y)
                scores[(column1, column2)] = score

        scores_sorted = dict( sorted(scores.items(), key=operator.itemgetter(1),reverse=True))
        best_feature_set = list(scores_sorted.keys())[0]
        best_feature1_values = np.unique(X[best_feature_set[0]])
        best_feature2_values = np.unique(X[best_feature_set[1]])

        best_feature1_values.sort()
        best_feature2_values.sort()

        partition = self.get_partition(X, Y, best_feature_set[0], best_feature_set[1], best_feature1_values, best_feature2_values)
        
        if partition == None:
            current_node.is_leaf = True
            unq, counts = np.unique(Y, return_counts=True)
            max_freq_idx = np.argmax(counts).flatten()
            current_node.value = unq[max_freq_idx].squeeze()
            current_node.name = f'leaf {current_node.value} {self.node_count}'
            self.node_count += 1
            return
            
        (X_left1, Y_left1), (X_left2, Y_left2), (X_right1, Y_right1), (X_right2, Y_right2), threshold1, threshold2 = partition

        threshold = [threshold1, threshold2]
        # print("Thresholds: ",threshold)
        current_node.threshold = threshold
        current_node.feature = best_feature_set
        current_node.name = f'{best_feature_set} {threshold} {self.node_count}'
        self.node_count += 1
        current_node.left_child1 = Node('unnamed')
        current_node.left_child2 = Node('unnamed')
        current_node.right_child1 = Node('unnamed')
        current_node.right_child2 = Node('unnamed')

        if(X_left1.shape[0] == 0):
            current_node.left_child1.is_leaf = True
            current_node.left_child1.value = 1
            current_node.left_child1.name = f'leaf {current_node.left_child1.value} {self.node_count}'
            self.node_count+=1 
            # return
        
        if(X_left2.shape[0] == 0):
            current_node.left_child2.is_leaf = True
            current_node.left_child2.value = 1
            current_node.left_child2.name = f'leaf {current_node.left_child2.value} {self.node_count}'
            self.node_count+=1 
            # return
        
        if(X_right1.shape[0] == 0):
            current_node.right_child1.is_leaf = True
            current_node.right_child1.value = 1
            current_node.right_child1.name = f'leaf {current_node.right_child1.value} {self.node_count}'
            self.node_count+=1   
            # return     
        
        if(X_right2.shape[0] == 0):
            current_node.right_child2.is_leaf = True
            current_node.right_child2.value = 1
            current_node.right_child2.name = f'leaf {current_node.right_child2.value} {self.node_count}'
            self.node_count+=1  
            # return                  
          
        if(X_left1.shape[0] != 0):
            self.fit_util(X_left1, Y_left1, current_node.left_child1)
        
        if(X_left2.shape[0] != 0):
            self.fit_util(X_left2, Y_left2, current_node.left_child2)
        
        if(X_right1.shape[0] != 0):
            self.fit_util(X_right1, Y_right1, current_node.right_child1)
        
        if(X_right2.shape[0] != 0):
            self.fit_util(X_right2, Y_right2, current_node.right_child2)


    def do_split(self, X, thresh):
        """
            Split the data at a node based on threshold
        """

        left_child_ids = np.where(X <= thresh, True, False)
        right_child_ids = np.where(X > thresh, True, False)
        return left_child_ids, right_child_ids

    def do_split_final(self, X1, X2, thresh1, thresh2):

        """
            Split according to the best thresholds for the 2 features
        """   

        left1_ids = np.where(np.logical_and(X1 <= thresh1, X2 <= thresh2), True, False)
        left2_ids = np.where(np.logical_and(X1 > thresh1, X2 <= thresh2), True, False)
        right1_ids = np.where(np.logical_and(X1 <= thresh1, X2 > thresh2), True, False)
        right2_ids = np.where(np.logical_and(X1 > thresh1, X2 > thresh2), True, False)

        return left1_ids, left2_ids, right1_ids, right2_ids

    def find_entropy(self, Y):
        probs = []
        possible_classes, counts = np.unique(Y, return_counts=True)
        sort_indices = np.argsort(possible_classes)
        possible_classes = possible_classes[sort_indices]
        counts = counts[sort_indices]
        
        for class_label, count in zip(possible_classes, counts):
            probs.append(count/Y.shape[0])
        
        entropy = 0
        for prob in probs:
            entropy -= prob*np.log2(prob)
        
        return entropy
    
    def find_best_thresh_info_gain(self, thresholds1, thresholds2, X, Y, feature1, feature2):
        """
            This function finds the best threshold and info gain for a given feature
        """
        best_info_gain = -float('inf')
        best_thresh1 = thresholds1[0]   
        best_thresh2 = thresholds2[0]     
        for thresh1 in thresholds1[:-1]:
            for thresh2 in thresholds2[:-1]:

                left_child1_ids, left_child2_ids,  right_child1_ids, right_child2_ids = self.do_split_final(X[feature1].to_numpy(),X[feature2].to_numpy(), thresh1, thresh2)

                parent_pts = X.shape[0]
                left_child1_pts = len(left_child1_ids)
                left_child2_pts = len(left_child2_ids)
                right_child1_pts = len(right_child1_ids)
                right_child2_pts = len(right_child2_ids)

                info_gain = self.find_entropy(Y) - (left_child1_pts / parent_pts) * self.find_entropy(Y[left_child1_ids]) - (left_child2_pts / parent_pts) * self.find_entropy(Y[left_child2_ids]) - (right_child1_pts / parent_pts) * self.find_entropy(Y[right_child1_ids]) - (right_child2_pts / parent_pts) * self.find_entropy(Y[right_child2_ids])
                
                if(info_gain > best_info_gain):

                    best_info_gain = info_gain
                    best_thresh1 = thresh1
                    best_thresh2 = thresh2

        
        return best_thresh1, best_thresh2, best_info_gain

    def get_partition(self, X, Y, feature1, feature2, thresholds1, thresholds2):
        '''
            This function should return left and right
            partitions according to appropritate
            partitioning algorithm. Return None if all
            data has same label
        '''
        
        # if only 1 class available at a node OR MIN_SAMPLES left at a node - Leaf Node reached
        if(len(Y) < self.min_samples):
            return None
        
        best_thresh1, best_thresh2, best_info_gain = self.find_best_thresh_info_gain(thresholds1, thresholds2, X, Y, feature1, feature2)

        # partition according to best threshold
        best_left_ids1, best_left_ids2, best_right_ids1, best_right_ids2 = self.do_split_final(X[feature1].to_numpy(), X[feature2].to_numpy(), best_thresh1, best_thresh2)

        # print(len(Y[best_left_ids1]), len(Y[best_left_ids2]), len(Y[best_right_ids1]) , len(Y[best_right_ids2]))

        count_0 = 0
        if(len(Y[best_left_ids1]) == 0):
            count_0 += 1
        if(len(Y[best_left_ids2]) == 0):
            count_0 += 1
        if(len(Y[best_right_ids1]) == 0):
            count_0 += 1
        if(len(Y[best_right_ids2]) == 0):
            count_0 += 1

        if(count_0 == 3):
            print(len(Y[best_left_ids1]), len(Y[best_left_ids2]), len(Y[best_right_ids1]) , len(Y[best_right_ids2]))
            return None

        return (X[best_left_ids1], Y[best_left_ids1]), (X[best_left_ids2], Y[best_left_ids2]), (X[best_right_ids1], Y[best_right_ids1]),(X[best_right_ids2], Y[best_right_ids2]), best_thresh1, best_thresh2    
    
    def print_tree(self):
        tree = Tree()
        self.print_tree_util(self.root_node, tree)
        tree.show()
        return tree

    def print_tree_util(self, root, tree, parent=None):
        if parent is not None:
            print(root.name)
            tree.create_node(root.name, root.name, parent=parent.name)
        else:
            print(root.name)
            tree.create_node(root.name, root.name)
        if root.is_leaf:
            return
        self.print_tree_util(root.left_child1, tree, root)
        self.print_tree_util(root.left_child2, tree, root)
        self.print_tree_util(root.right_child1, tree, root)
        self.print_tree_util(root.right_child2, tree, root)


In [83]:
iris = load_iris()
X_iris, Y_iris = iris.data, iris.target
X_iris_train,X_iris_test,Y_iris_train,Y_iris_test=train_test_split(X_iris,Y_iris,test_size=0.2,random_state=42)
X_iris_train = pd.DataFrame(X_iris_train)
X_iris_test = pd.DataFrame(X_iris_test)

print(X_iris_train.shape)
print(X_iris_test.shape)

# X_iris = pd.DataFrame(X_iris)
# print(X_iris.shape)

(120, 4)
(30, 4)


In [84]:
dt_iris = MyDecisionTree(min_samples=110)
dt_iris.fit(X_iris_train, Y_iris_train)
tree = dt_iris.print_tree()

(2, 3) [1.9, 1.7] 0
leaf 0 2
leaf 1 3
leaf 1 1
leaf 2 4
(2, 3) [1.9, 1.7] 0
├── leaf 0 2
├── leaf 1 1
├── leaf 1 3
└── leaf 2 4



In [85]:
tree.to_graphviz() 

digraph tree {
	"(2, 3) [1.9, 1.7] 0" [label="(2, 3) [1.9, 1.7] 0", shape=circle]
	"leaf 0 2" [label="leaf 0 2", shape=circle]
	"leaf 1 1" [label="leaf 1 1", shape=circle]
	"leaf 1 3" [label="leaf 1 3", shape=circle]
	"leaf 2 4" [label="leaf 2 4", shape=circle]

	"(2, 3) [1.9, 1.7] 0" -> "leaf 0 2"
	"(2, 3) [1.9, 1.7] 0" -> "leaf 1 3"
	"(2, 3) [1.9, 1.7] 0" -> "leaf 1 1"
	"(2, 3) [1.9, 1.7] 0" -> "leaf 2 4"
}


In [86]:
pred_Y = dt_iris.predict(X_iris_test.to_numpy())    

In [87]:
print(pred_Y, Y_iris_test)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0] [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [88]:
accuracy = (sum(1 for x,y in zip(pred_Y, Y_iris_test) if x == y) / len(pred_Y))*100
accuracy

100.0

In [89]:
digits = load_digits()
X_digits, Y_digits = digits.data, digits.target
X_digits = X_digits/255
X_digits_train,X_digits_test,Y_digits_train,Y_digits_test=train_test_split(X_digits,Y_digits,test_size=0.2,random_state=42)
X_digits_train = pd.DataFrame(X_digits_train)
X_digits_test = pd.DataFrame(X_digits_test)

print(X_digits_train.shape)
print(X_digits_test.shape)

# X_iris = pd.DataFrame(X_iris)
# print(X_iris.shape)

(1437, 64)
(360, 64)


In [90]:
dt_digits = MyDecisionTree(min_samples=1)
dt_digits.fit(X_digits_train, Y_digits_train)
tree = dt_digits.print_tree()

290 0 0 0
71 0 0 0
110 0 0 0
500 0 0 0
236 0 0 0
6 0 0 0
91 0 0 0
94 0 0 0
(21, 42) [0.0, 0.058823529411764705] 0
leaf 5 1
(30, 60) [0.054901960784313725, 0.0] 2
leaf 7 4
leaf 1 3
(28, 36) [0.0, 0.058823529411764705] 5
leaf 0 6
leaf 9 7
leaf 7 8
leaf 1 9
leaf 4 10
leaf 6 11
(36, 36) [0.0, 0.0] 12
leaf 0 15
leaf 1 13
leaf 1 14
leaf 4 16
(21, 42) [0.0, 0.058823529411764705] 0
├── (30, 60) [0.054901960784313725, 0.0] 2
│   ├── (28, 36) [0.0, 0.058823529411764705] 5
│   │   ├── leaf 0 6
│   │   ├── leaf 1 9
│   │   ├── leaf 7 8
│   │   └── leaf 9 7
│   ├── leaf 1 3
│   ├── leaf 4 10
│   └── leaf 7 4
├── (36, 36) [0.0, 0.0] 12
│   ├── leaf 0 15
│   ├── leaf 1 13
│   ├── leaf 1 14
│   └── leaf 4 16
├── leaf 5 1
└── leaf 6 11



In [92]:
tree.to_graphviz()

digraph tree {
	"(21, 42) [0.0, 0.058823529411764705] 0" [label="(21, 42) [0.0, 0.058823529411764705] 0", shape=circle]
	"(30, 60) [0.054901960784313725, 0.0] 2" [label="(30, 60) [0.054901960784313725, 0.0] 2", shape=circle]
	"(36, 36) [0.0, 0.0] 12" [label="(36, 36) [0.0, 0.0] 12", shape=circle]
	"leaf 5 1" [label="leaf 5 1", shape=circle]
	"leaf 6 11" [label="leaf 6 11", shape=circle]
	"(28, 36) [0.0, 0.058823529411764705] 5" [label="(28, 36) [0.0, 0.058823529411764705] 5", shape=circle]
	"leaf 1 3" [label="leaf 1 3", shape=circle]
	"leaf 4 10" [label="leaf 4 10", shape=circle]
	"leaf 7 4" [label="leaf 7 4", shape=circle]
	"leaf 0 15" [label="leaf 0 15", shape=circle]
	"leaf 1 13" [label="leaf 1 13", shape=circle]
	"leaf 1 14" [label="leaf 1 14", shape=circle]
	"leaf 4 16" [label="leaf 4 16", shape=circle]
	"leaf 0 6" [label="leaf 0 6", shape=circle]
	"leaf 1 9" [label="leaf 1 9", shape=circle]
	"leaf 7 8" [label="leaf 7 8", shape=circle]
	"leaf 9 7" [label="leaf 9 7", shape=circle]


In [91]:
pred_y_digits = dt_digits.predict(X_digits_test.to_numpy()).squeeze()
# print(pred_y_digits)

accuracy = (pred_y_digits == Y_digits_test).sum() / Y_digits_test.shape[0]
print(accuracy)

0.4722222222222222
